# LangGraph Code Walkthrough

This notebook lives next to the implementation and helps review actual runtime code under `execution/langgraph/`.

## Run Order

Restart kernel after code edits, then run cells top-to-bottom.

In [ ]:
from pathlib import Path
import sys

def _candidate_roots() -> list[Path]:
    cwd = Path.cwd().resolve()
    candidates = [cwd, *cwd.parents, Path('/home/nir/dev/agent_phase0')]
    seen = set()
    unique = []
    for c in candidates:
        if str(c) in seen:
            continue
        seen.add(str(c))
        unique.append(c)
    return unique

repo_root = None
for candidate in _candidate_roots():
    if (candidate / 'execution' / 'langgraph' / 'graph.py').exists():
        repo_root = candidate
        break

if repo_root is None:
    raise RuntimeError('Could not locate repo root containing execution/langgraph/graph.py')

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

import inspect
from execution.langgraph import graph, memo_store, policy, checkpoint_store
print('repo_root =', repo_root)

## Source Snapshot

In [ ]:
print('--- MemoizationPolicy ---')
print(inspect.getsource(policy.MemoizationPolicy))
print('--- _execute_action (duplicate + memo enforcement path) ---')
print(inspect.getsource(graph.LangGraphOrchestrator._execute_action))

## Instantiate Core Objects

In [ ]:
from execution.langgraph.memo_store import SQLiteMemoStore
from execution.langgraph.checkpoint_store import SQLiteCheckpointStore
from execution.langgraph.policy import MemoizationPolicy

memo = SQLiteMemoStore('.tmp/memo_store.db')
checkpoints = SQLiteCheckpointStore('.tmp/langgraph_checkpoints.db')
memo_policy = MemoizationPolicy(max_policy_retries=2)
memo, checkpoints, memo_policy

## Run With Scripted Provider (Offline)

In [ ]:
import json
from execution.langgraph.graph import LangGraphOrchestrator

class ScriptedProvider:
    def __init__(self, outputs):
        self.outputs = [json.dumps(item) for item in outputs]
        self.index = 0

    def generate(self, messages):
        if self.index < len(self.outputs):
            value = self.outputs[self.index]
            self.index += 1
            return value
        return self.outputs[-1]

provider_mock = ScriptedProvider([
    {'action': 'tool', 'tool_name': 'write_file', 'args': {'path': 'fib.txt', 'content': ','.join(str(i) for i in range(120))}},
    {'action': 'tool', 'tool_name': 'memoize', 'args': {'key': 'write_file:fib.txt', 'value': {'ok': True}, 'source_tool': 'write_file'}},
    {'action': 'finish', 'answer': 'done'}
])

orch = LangGraphOrchestrator(provider=provider_mock, memo_store=memo, checkpoint_store=checkpoints, policy=memo_policy)
result = orch.run('offline test')
result['answer']

## Inspect Counters and Checkpoints

In [ ]:
{
    'retry_counts': result['state']['retry_counts'],
    'tool_call_counts': result['state']['tool_call_counts'],
    'memo_events': result['memo_events'],
    'checkpoint_count': len(result['checkpoints']),
}

## Duplicate Tool Guard Demo

In [ ]:
duplicate_provider = ScriptedProvider([
    {'action': 'tool', 'tool_name': 'sort_array', 'args': {'items': [3, 2, 1], 'order': 'asc'}},
    {'action': 'tool', 'tool_name': 'sort_array', 'args': {'items': [3, 2, 1], 'order': 'asc'}},
    {'action': 'finish', 'answer': 'done'}
])
dup_orch = LangGraphOrchestrator(provider=duplicate_provider, memo_store=memo, checkpoint_store=checkpoints, policy=memo_policy)
dup_result = dup_orch.run('duplicate guard test')
{
    'tools_used': [x['tool'] for x in dup_result['tools_used']],
    'duplicate_tool_retries': dup_result['state']['retry_counts']['duplicate_tool'],
}